In [87]:
import pandas as pd
import re
from langdetect import detect

# Đọc file dữ liệu
books = pd.read_csv('dataset/books.csv')
ratings = pd.read_csv('dataset/ratings.csv')

print(ratings.head())

                 User_id  book_id  review/score
0  A00109803PZJ91RLT7DPN   134046           5.0
1  A00117421L76WVWG4UX95    84912           5.0
2   A0015610VMNR0JC9XVL1    51730           5.0
3   A0015610VMNR0JC9XVL1    65628           5.0
4   A0015610VMNR0JC9XVL1   105456           5.0


In [88]:
# Tính số lượng của mỗi User_id
user_rating_counts = ratings['User_id'].value_counts()

# Lọc ra những User_id có số đánh giá > 20
users_with_more_than_20_ratings = user_rating_counts[user_rating_counts > 50].index

# Lọc ratings chỉ giữ lại những hàng có User_id nằm trong danh sách trên
filtered_ratings = ratings[ratings['User_id'].isin(users_with_more_than_20_ratings)]
ratings = filtered_ratings
print(ratings)

               User_id  book_id  review/score
2811     A106016KSI0YQ      528           5.0
2812     A106016KSI0YQ     2457           4.0
2813     A106016KSI0YQ     3407           2.0
2814     A106016KSI0YQ     4191           3.0
2815     A106016KSI0YQ     6609           4.0
...                ...      ...           ...
2113015  AZSV99SDJC242   201547           4.0
2113016  AZSV99SDJC242   201753           5.0
2113017  AZSV99SDJC242   205200           4.0
2113018  AZSV99SDJC242   205262           4.0
2113019  AZSV99SDJC242   207728           4.0

[253226 rows x 3 columns]


In [89]:
import pandas as pd
import re

books = books.sample(n=5000, random_state=2)

ratings = ratings.merge(books[['book_id']], on='book_id', how='inner')

print(books)
print(ratings)

        book_id                                              Title  \
78458    117174  The two thousand isles;: A short account of th...   
41039     56697                        Noise control for engineers   
117463   171425  The Godless and the Infidels: Memoirs of a Ras...   
79366    118463  Fetish: Masterpieces of Erotic Fantasy Photogr...   
5006       7061     Raw Faith: Nurturing the Believer in All of Us   
...         ...                                                ...   
24708     33949  Lincoln and the Bluegrass: Slavery and Civil W...   
122861   181825  Musical Qigong: Ancient Chinese Healing Art fr...   
60028     90097                                   The Perfect Ride   
58215     86321                The Infamous Rake (Regency Romance)   
18269     25314                              Breakfasts & Brunches   

                                              description  \
78458   a short and classic account of the authors tra...   
41039   this classic and authoritativ

In [90]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu ratings thành hai tập train và test
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

# Kiểm tra kích thước của mỗi tập
print("Kích thước tập train:", train.shape)
print("Kích thước tập test:", test.shape)


Kích thước tập train: (5622, 3)
Kích thước tập test: (1406, 3)


In [91]:
ratings = train

In [92]:
all_book_ids = ratings['book_id'].unique().tolist()

print(len(all_book_ids))

1676


In [93]:
ratings['review/score'] = pd.to_numeric(ratings['review/score'], errors='coerce')
ratings = ratings[ratings['User_id'].astype(str).str.strip() != '']
ratings = ratings[ratings['User_id'].astype(str).str.strip() != '']

ratings = ratings.groupby(['User_id', 'book_id'], as_index=False)['review/score'].mean()


In [94]:
import pandas as pd

def create_user_item_matrix(books_rating_df):
    user_item_matrix = books_rating_df.pivot(index='book_id', columns='User_id', values='review/score')

    return user_item_matrix

user_item_matrix = create_user_item_matrix(ratings)

print("Ma trận đánh giá ban đầu:")
print(user_item_matrix)


Ma trận đánh giá ban đầu:
User_id  A106016KSI0YQ  A106E1N0ZQ4D9W  A107C4RVRF0OP  A10EH95EUDO9W3  \
book_id                                                                 
68                 NaN             NaN            NaN             NaN   
99                 NaN             NaN            NaN             NaN   
275                NaN             NaN            NaN             NaN   
293                NaN             NaN            NaN             NaN   
428                NaN             NaN            NaN             NaN   
...                ...             ...            ...             ...   
211031             NaN             NaN            NaN             NaN   
211126             NaN             NaN            NaN             NaN   
211876             NaN             NaN            NaN             NaN   
212042             NaN             NaN            NaN             NaN   
212049             NaN             NaN            NaN             NaN   

User_id  A10HWJXUG67E7I 

In [95]:
print(user_item_matrix.index)

Index([    68,     99,    275,    293,    428,    440,    598,    825,   1121,
         1200,
       ...
       210027, 210451, 210516, 210659, 210677, 211031, 211126, 211876, 212042,
       212049],
      dtype='int64', name='book_id', length=1676)


In [96]:
import pandas as pd

def normalize_rating_matrix_by_item(user_item_matrix):
    item_mean = user_item_matrix.mean(axis=1, skipna=True)
    
    normalized_user_item_matrix = user_item_matrix.sub(item_mean, axis=0)
    
    return normalized_user_item_matrix, item_mean

normalized_user_item_matrix, item_mean  = normalize_rating_matrix_by_item(user_item_matrix)

print("\nMa trận đánh giá đã chuẩn hóa theo item mean:")
print(normalized_user_item_matrix)



Ma trận đánh giá đã chuẩn hóa theo item mean:
User_id  A106016KSI0YQ  A106E1N0ZQ4D9W  A107C4RVRF0OP  A10EH95EUDO9W3  \
book_id                                                                 
68                 NaN             NaN            NaN             NaN   
99                 NaN             NaN            NaN             NaN   
275                NaN             NaN            NaN             NaN   
293                NaN             NaN            NaN             NaN   
428                NaN             NaN            NaN             NaN   
...                ...             ...            ...             ...   
211031             NaN             NaN            NaN             NaN   
211126             NaN             NaN            NaN             NaN   
211876             NaN             NaN            NaN             NaN   
212042             NaN             NaN            NaN             NaN   
212049             NaN             NaN            NaN             NaN   

Use

In [97]:
# Tạo dictionary ánh xạ book_id đến giá trị trung bình
item_mean_dict = item_mean.to_dict()

# Hàm để lấy giá trị trung bình của một book_id từ item_mean_dict
def get_item_mean(book_id, item_mean_dict):
    return item_mean_dict.get(book_id, None)  # Dùng get để trả về None nếu không tìm thấy

# Ví dụ, lấy giá trị trung bình của book_id = 68
book_id = 428
item_mean_value = get_item_mean(book_id, item_mean_dict)

print(f"Giá trị trung bình của Book ID {book_id}: {item_mean_value}")

Giá trị trung bình của Book ID 428: 5.0


In [98]:
def get_rating_counts_for_book(user_item_matrix, book_id):
    if book_id not in user_item_matrix.index:
        print(f"Book ID {book_id} không tồn tại trong dữ liệu.")
        return []
    
    # Lấy các đánh giá của sách theo book_id (cột)
    ratings = user_item_matrix.loc[book_id]
    
    # Thay thế NaN bằng giá trị 0 hoặc có thể thay bằng "Chưa đánh giá"
    ratings_filled = ratings.fillna(0)
    
    # Đếm số lượng đánh giá cho từng giá trị
    rating_counts = ratings_filled.value_counts().sort_index()
    
    # In ra các kết quả
    print(f"\nĐánh giá của Book ID {book_id}:")
    for rating, count in rating_counts.items():
        print(f"Rating: {rating}, Số lượng: {count}")
    
    return rating_counts

# Lấy các đánh giá và đếm số lượng cho book_id = 68
rating_counts_for_book_68 = get_rating_counts_for_book(user_item_matrix, 2961)


Đánh giá của Book ID 2961:
Rating: 0.0, Số lượng: 1772
Rating: 5.0, Số lượng: 1


In [99]:
print(all_book_ids)

[64901, 113076, 108150, 211126, 33280, 58523, 63555, 189892, 126224, 83393, 105543, 179381, 55110, 161366, 208825, 75374, 118244, 51644, 181954, 150428, 137358, 132203, 154232, 89961, 39422, 106917, 116385, 156003, 146653, 102051, 196705, 133211, 78571, 183762, 52394, 122919, 84787, 31035, 91982, 38627, 104977, 192508, 61144, 57661, 4422, 150123, 167950, 201019, 31957, 40577, 192586, 59670, 44937, 105893, 125658, 193301, 168693, 184633, 85619, 14722, 102047, 145986, 69267, 57624, 5893, 158510, 38127, 104120, 45116, 53681, 8388, 142973, 153723, 50689, 163590, 4859, 142880, 183959, 57195, 151628, 42356, 195333, 179461, 144988, 160757, 108379, 16950, 48053, 123295, 123913, 56045, 136874, 62420, 180773, 90485, 187247, 188468, 191273, 13519, 21933, 183359, 157841, 134948, 23409, 64401, 2931, 190407, 190685, 77441, 78040, 156188, 144348, 56457, 181999, 90471, 207308, 38066, 29287, 197676, 105302, 146835, 5382, 174699, 63000, 185128, 89187, 165036, 115305, 96079, 197418, 14215, 69587, 203657,

In [100]:
# import pandas as pd
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# # Giả sử normalized_user_item_matrix là ma trận user-item với users là hàng và items là cột

# # Điền các giá trị NaN với 0 (hoặc bạn có thể chọn phương pháp khác)
# filled_user_item_matrix = normalized_user_item_matrix.fillna(0)

# # Chuyển vị để có ma trận item-user (items là hàng, users là cột)
# item_user_matrix = filled_user_item_matrix

# # Tính ma trận cosine similarity giữa các items
# similarity_matrix = cosine_similarity(item_user_matrix)

# # Tạo DataFrame để dễ xem kết quả
# similarity_df = pd.DataFrame(similarity_matrix, index=item_user_matrix.index, columns=item_user_matrix.index)

# print("Ma trận tương đồng giữa các sách (items) bằng cosine similarity (sau khi điền NaN):")
# print(similarity_df)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
similarity_df = normalized_user_item_matrix.T.corr(method='pearson')
print("Ma trận tương đồng giữa các sách (items):")
print(similarity_df)


In [ ]:
import pandas as pd
import numpy as np

def recommend_books(user_id, top_k=5):

    # Lấy các sách đã được đánh giá
    rated_books = ratings[ratings['User_id'] == user_id]['book_id'].unique()
    
    if len(rated_books) == 0:
        print(f"Người dùng {user_id} chưa đánh giá sách nào.")
        return []
    
    # Lấy các sách đã được người dùng đánh giá
    user_ratings = user_item_matrix.get(user_id, pd.Series())
    normalized_user_ratings = normalized_user_item_matrix.get(user_id, pd.Series())
    
    # Khởi tạo một từ điển để lưu điểm gợi ý
    recommendation_scores = {}
     
    for book_id in all_book_ids:
        if(book_id not in rated_books):
            # Lấy độ tương đồng của sách chưa được đánh giá với các sách đã được đánh giá
            similar_books = similarity_df[book_id].dropna()
            # Tính weighted_sum và total_similarity
            weighted_sum = 0
            total_similarity = 0
        
            for rated_book in rated_books:
                if rated_book in similar_books.index:
                    weighted_sum += (similar_books[rated_book] * normalized_user_ratings[rated_book])
                    total_similarity += abs(similar_books[rated_book])
        
            # Tránh chia cho 0
            if total_similarity != 0:
                recommendation_scores[book_id] = (weighted_sum / total_similarity) + get_item_mean(book_id, item_mean_dict)
    
    # Sắp xếp các sách gợi ý theo điểm và chọn n_recommendations
    recommended_books = sorted(recommendation_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
    
    # Kiểm tra nếu không có sách nào được gợi ý
    if not recommended_books:
        return []
    
    return recommended_books


In [ ]:
user_id = "AZSV99SDJC242"
top_k = 10

suggested_books = recommend_books(user_id, top_k)
for book_id, similarity_score in suggested_books:
    print(book_id, end=" ")  # Print each book_id followed by a 
    
print()
if suggested_books:
    for book_id, similarity_score in suggested_books:
        # Fetch book details from the books DataFrame
        book_info = books.loc[books['book_id'] == book_id, ['Title', 'categories', 'description', 'authors']]
        
        if not book_info.empty:
            title = book_info['Title'].values[0]
            categories = book_info['categories'].values[0]
            description = book_info['description'].values[0]
            authors = book_info['authors'].values[0]
            
            # Print book information
            print(f"Sách: '{title}' (ID: {book_id})")
            print(f"Thể loại: {categories}")
            print(f"Tác giả: {authors}")
            print(f"Mô tả: {description}")
            print(f"Điểm tương đồng: {similarity_score:.4f}")
            print("-" * 50)
        else:
            print(f"Không tìm thấy sách với ID: {book_id}")
else:
    print("Không có cuốn sách nào để gợi ý.")


146835 122919 89961 72092 180493 127946 116385 133211 81085 192586 
Sách: 'Stolen (Women of the Otherworld, Book 2)' (ID: 146835)
Thể loại: ['Fiction']
Tác giả: ['Kelley Armstrong']
Mô tả: the only living female werewolf thirtyyearold elena michaels takes on a mission to confront internet billionaire tyrone winsloe who has been capturing elenas fellow paranormal brethren conducting experiments on them and hunting them to their deaths in a live video game by the author of bitten reprint
Điểm tương đồng: 5.2601
--------------------------------------------------
Sách: 'Post Captain (Master and Commander Series)' (ID: 122919)
Thể loại: ['Fiction']
Tác giả: ['"Patrick OBrian"']
Mô tả: if jane austen had written rousing sea yarns she would have produced something very close to the prose of patrick obrian time its 1802 the treaty of amiens has brought an end to the hostilities between great britain and france captain jack aubrey and his friend naval surgeon stephen maturin are enjoying the re

In [ ]:
# In ra tất cả giá trị đánh giá cho một book_id cụ thể, ví dụ: book_id = 68
book_id = 119048
book_ratings = similarity_df.loc[book_id].dropna()
print(f"Tất cả giá trị đánh giá cho book_id {book_id}:")
print(book_ratings)


Tất cả giá trị đánh giá cho book_id 119048:
book_id
119048    1.0
Name: 119048, dtype: float64


In [ ]:
# In ra tất cả giá trị đánh giá cho một book_id cụ thể, ví dụ: book_id = 68
book_id = 119048
book_ratings = normalized_user_item_matrix.loc[book_id].dropna()
print(f"Tất cả giá trị đánh giá cho book_id {book_id}:")
print(book_ratings)


Tất cả giá trị đánh giá cho book_id 119048:
User_id
A2VE83MZF98ITY    1.5
A68QXK0H88J4I    -1.5
Name: 119048, dtype: float64


In [ ]:
def get_books_for_user(user_id):
    # Filter ratings to get books rated by the specified user_id
    user_ratings = ratings[ratings['User_id'] == user_id]
    
    # Merge with books DataFrame to get book details including title, categories, description, and authors
    user_books = user_ratings.merge(books[['book_id']], on='book_id', how='inner')
    
    # Return the list of book_ids
    book_ids = user_books['book_id'].tolist()
    
    return book_ids

# Example usage
user_id = "A2VE83MZF98ITY"
list_books = get_books_for_user(user_id)
print(list_books)
if list_books:  # Ensure that the list of book_ids is not empty
    for book_id in list_books:
        # Fetch book details from the books DataFrame
        book_info = books.loc[books['book_id'] == book_id, ['Title', 'categories', 'description', 'authors']]
        
        if not book_info.empty:
            title = book_info['Title'].values[0]
            categories = book_info['categories'].values[0]
            description = book_info['description'].values[0]
            authors = book_info['authors'].values[0]
            
            # Print book information
            print(f"Sách: '{title}' (ID: {book_id})")
            print(f"Thể loại: {categories}")
            print(f"Tác giả: {authors}")
            print(f"Mô tả: {description}")
            print("-" * 50)
        else:
            print(f"Không tìm thấy sách với ID: {book_id}")
else:
    print("Không có cuốn sách nào để gợi ý.")


[29116, 30757, 46350, 47366, 95129, 114632, 119048, 156003, 160061, 187352, 188062, 190407]
Sách: 'The Greeks' (ID: 29116)
Thể loại: ['History']
Tác giả: ['Roderick Beaton']
Mô tả: a sweeping history of the greeks from the bronze age to today more than two thousand years ago the greek citystates led by athens and sparta laid the foundation for much of modern science the arts politics and law but the influence of the greeks did not end with the rise and fall of this classical civilization as historian roderick beaton illustrates over three millennia greek speakers produced a series of civilizations that were rooted in southeastern europe but again and again ranged widely across the globe in the greeks beaton traces this history from the bronze age mycenaeans who built powerful fortresses at home and strong trade routes abroad to the dramatic eurasian conquests of alexander the great to the pious byzantines who sought to export christianity worldwide to todays greek diaspora which flouri

In [ ]:
# Lấy danh sách UserID độc nhất
user_ids = test['User_id'].unique().tolist()

print("Danh sách UserID độc nhất:")
print(len(user_ids))


Danh sách UserID độc nhất:
894


In [ ]:
import random

# Lấy ngẫu nhiên 1000 UserID từ danh sách
user_ids_sample = random.sample(user_ids, 500)
user_ids = user_ids_sample
print("Danh sách 1000 UserID ngẫu nhiên:")
print(user_ids)


Danh sách 1000 UserID ngẫu nhiên:
['AW7BIYHXUIZ62', 'A3C0380R9NMLHD', 'A1C58YXM2A30CY', 'A1ZOF747O5EB6R', 'A3TEH90X39WC8F', 'A3B48AN8JID1JK', 'AF3X7J0XC391L', 'A1PPD5TOR6VVYV', 'A3B1QVMBW7YVOQ', 'AKB2PYODH0TKS', 'A3MBJ9TIYG91OL', 'AQZH7YTWQPOBE', 'A1JTG5X4VHJV27', 'A38VAF4S1DARTN', 'A3J0OXB9KIC5SS', 'A1D5RCOILPC9LX', 'A1TMAVN4CEM8U8', 'AFVQZQ8PW0L', 'A222LQEPE7O7BV', 'A2JA7C0WEMB1M7', 'AG2IEP1MJQHFS', 'A1I2O9Y3X3HXLS', 'A9RUU8JSKU7G1', 'A3CU3GER3D7K7A', 'A1K1JW1C5CUSUZ', 'A2ZHDCYYP1PL2K', 'A1G0DH7UQ4ILS6', 'A4P7HYSOIVBLL', 'A1NPNGWBVD9AK3', 'A2QT0KPQU671OU', 'A1XOHJLA16DPZT', 'A2B9Y0WXNSN17U', 'ADB8XCKNSDY5Z', 'A32ZKBXJJ45BRY', 'A17FLA8HQOFVIG', 'ATOFFO1VNCAGQ', 'A5Y1PJIOU21WD', 'A2WB4OWBUH2VQX', 'A1RLMYWYWQ03QJ', 'A3E8QNDC7CV44', 'A2R504QEK5URRL', 'A29LGHDM4PTVOA', 'A2EMP366TTS6E1', 'A1DPCXO7RDZM0S', 'AKHJVOYEX6QME', 'A2CR57GAJKNWVV', 'A3TO3AY1UFM83U', 'AD8UQ909Z49UK', 'A2X9QA4JU5Q4VY', 'A2VZ11U5DXM8J5', 'A2JXAQ92WYPAAR', 'A20MN959ZZH2DU', 'AOV9AIX4UE5BB', 'A136YD08SCJ2LV', 'A2MOBMVHE

In [ ]:
import numpy as np

total_tp = 0  # Tổng số True Positives
total_fp = 0  # Tổng số False Positives
total_fn = 0  # Tổng số False Negatives
total_squared_error = 0  # Tổng bình phương sai số
total_absolute_error = 0  # Tổng sai số tuyệt đối
total_predictions = 0  # Tổng số dự đoán được sử dụng cho MSE, RMSE, MAE

count = 0

for user_id in user_ids:
    count +=1
    print(f"Bắt đầu tính user thứ {count}")
    actual_books = test[test['User_id'] == user_id][['book_id', 'review/score']].values.tolist()
    
    suggested_books = recommend_books(user_id, top_k=10) 
    
    actual_dict = dict(actual_books)
    
    suggested_book_ids = set([book_id for book_id, _ in suggested_books])
    actual_book_ids = set([book_id for book_id, _ in actual_books])

    # TP (True Positives), FP (False Positives) và FN (False Negatives) cho use hiện tại
    tp = len(suggested_book_ids.intersection(actual_book_ids))
    fp = len(suggested_book_ids - actual_book_ids)
    fn = len(actual_book_ids - suggested_book_ids)
    
    total_tp += tp
    total_fp += fp
    total_fn += fn

    # MSE, RMSE, MAE
    for book_id, predicted_score in suggested_books:
        if book_id in actual_dict:
            actual_score = actual_dict[book_id]
            total_squared_error += (actual_score - predicted_score) ** 2
            total_absolute_error += abs(actual_score - predicted_score)
            total_predictions += 1

precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

mse = total_squared_error / total_predictions if total_predictions > 0 else 0
rmse = np.sqrt(mse)
mae = total_absolute_error / total_predictions if total_predictions > 0 else 0

print("Overall Precision:", precision)
print("Overall Recall:", recall)
print("Overall F1-score:", f1_score)
print("Overall MSE:", mse) 
print("Overall RMSE:", rmse)
print("Overall MAE:", mae)


Đang tính user 1


KeyError: 'UserID'